In [1]:
import glob
import matplotlib.pyplot as plt
import pandas as pd

from ipywidgets import interact, Dropdown

---

In [2]:
xls = pd.ExcelFile('estabelecimentos/layout.xlsx')

In [3]:
dt_sections = pd.read_excel(xls, 'seção').set_index('Código').to_dict()['Descrição']

---

In [4]:
df = pd.DataFrame()

In [5]:
for csv in glob.glob('estabelecimentos/files/*csv'):
    data = pd.read_csv(csv, sep = ';')
    df = df.append(data)

In [6]:
cols = [
    'região',
    'uf',
    'fonte_desl', 
    'tipoempregador', 
    'tipoestabelecimento', 
    'tamestabjan'
]

In [7]:
df = df.drop(cols, axis = 1)

In [8]:
df = df.rename(
    columns = {
        'competência': 'competencia', 
        'município': 'municipio', 
        'seção': 'secao', 
        'saldomovimentação': 'saldo'
    }
)

In [9]:
df = df.loc[df['municipio'] == 352590]

---

In [10]:
def section_graph(section, df_balance):
    fig, ax = plt.subplots(figsize = (15, 10))

    plt.suptitle(f'Saldo de Empregos', fontsize = 20, fontweight = 'bold')
    plt.title(section, fontsize = 18, y = 1.035)

    graph = ax.bar(df_balance['competencia'].values, df_balance['saldo'].values, zorder = 2)

    for bar in graph:
        height = bar.get_height()
        y_pos = height * 1.035
        
        if height >= 0:
            v_align = 'bottom'
        else:
            v_align = 'top'

        ax.text(bar.get_x() + bar.get_width() / 2, y_pos, height, ha = 'center', va = v_align, fontweight = 'bold')

    plt.grid(axis = 'y', zorder = 0)
    plt.show()

In [11]:
def get_balance(section):
    df_balance = df.loc[df['secao'] == section].groupby('competencia').agg({'saldo': 'sum'}).reset_index()
    df_balance['competencia'] = df_balance['competencia'].astype(str)
    df_balance['competencia'] = df_balance['competencia'].apply(lambda x: x[4:] + '/' + x[2:4])
    
    return df_balance

---

In [12]:
dd_section = Dropdown(options = dt_sections.values())

In [13]:
@interact(section = dd_section)
def picker(section):
    section_code = list(dt_sections.keys())[list(dt_sections.values()).index(section)]
    
    balance = get_balance(section_code)
    section_graph(section, balance)

interactive(children=(Dropdown(description='section', options=('Agricultura, Pecuária, Produção Florestal, Pes…